In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [47]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [48]:
# Load data
train = pd.read_csv("../input/train.csv").drop('id', axis=1)
test = pd.read_csv('../input/test.csv')

In [49]:
# Manage data on train set
y_train = train['target']
X_train = train.drop('target', axis=1)

# Manage data on test set
X_test = test.drop('id', axis = 1)

In [51]:
# Normalize data
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.fit_transform(X_test)

In [52]:
# GridSearchCV with XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)
param_grid = {'objective':['binary:logistic'],
              'learning_rate': [0.001,0.05,0.1, 10], 
              'max_depth': [2,3,4,5,6],
              'min_child_weight': [11],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [1000]}

grid = GridSearchCV(estimator = xgb_model, cv=5, param_grid = param_grid , scoring = 'roc_auc', verbose = 1, n_jobs = -1, refit=True)
grid.fit(X_train,y_train)

print("Best Score:" + str(grid.best_score_))
print("Best Parameters: " + str(grid.best_params_))

best_parameters = grid.best_params_

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   56.9s finished


Best Score:0.7704861111111112
Best Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 11, 'n_estimators': 1000, 'objective': 'binary:logistic', 'subsample': 0.8}


In [53]:
# XGBoost model with RFE and 70 features
xgb_model = xgb.XGBClassifier(**best_parameters)
xgb_model.fit(X_train,y_train)

selector = RFE(xgb_model, 200, step=1)
selector.fit(X_train,y_train)

xgb_preds = selector.predict_proba(X_test)[:,1]

train_predict = selector.predict(X_train)
roc_auc_score(y_train, train_predict)

1.0

In [54]:
# GridSearchCV with LogisticRegressor model
lr = LogisticRegression(random_state=42)
param_grid = {'class_weight' : ['balanced', None], 
              'penalty' : ['l2','l1'], 
              'C' : [0.001, 0.01, 0.1, 1, 10, 100],
              'solver': ['saga']}

grid = GridSearchCV(estimator = lr, cv=5, param_grid = param_grid , scoring = 'roc_auc', verbose = 1, n_jobs = -1)
grid.fit(X_train,y_train)

print("Best Score:" + str(grid.best_score_))
print("Best Parameters: " + str(grid.best_params_))

best_parameters = grid.best_params_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:0.8097222222222221
Best Parameters: {'C': 0.1, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'}


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    6.5s finished


In [55]:
# LogisticRegression with RFE and 150 features
lr = LogisticRegression(**best_parameters)
lr.fit(X_train,y_train)

selector = RFE(lr, 200, step=1)
selector.fit(X_train,y_train)

lr_preds = lr.predict_proba(X_test)[:,1]

train_predict = lr.predict(X_train)
roc_auc_score(y_train, train_predict)

0.8059027777777777

In [56]:
# Combine models
final_preds = (lr_preds * 0.8 + xgb_preds * 0.2)
submission = pd.read_csv('../input/sample_submission.csv')
submission['target'] = final_preds
submission.to_csv('submission.csv', index=False)
submission.head(20)

,id,target
0,250,0.771676
1,251,0.608105
2,252,0.629034
3,253,0.866624
4,254,0.567588
5,255,0.476844
6,256,0.530718
7,257,0.363034
8,258,0.852487
9,259,0.410239
